code start here

In [0]:
import requests
import cv2
import os

In [8]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
API_KEY = "4248579e1322494d8283ea682649f437"
PATH = '/content/drive/My Drive/FSDP2019_DEEP_LEARNING/day30andcodechallenge/dataset/'
MAX_RESULTS = 250
GROUP_SIZE = 50

In [0]:

if not os.path.isdir(PATH):
    os.mkdir(PATH)

# set the endpoint API URL
URL = "https://api.cognitive.microsoft.com/bing/v7.0/images/search/"


In [0]:
pokemons = []

while True:
    name = input("Enter pokemon Name: ").lower()
    if not name:
        break
    pokemons.append(name)


Enter pokemon Name: pikachu
Enter pokemon Name: 


In [0]:
pokemons

['pikachu']

In [3]:
# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 3, activation = 'softmax'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


Using TensorFlow backend.


In [5]:

# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/My Drive/FSDP2019_DEEP_LEARNING/day30andcodechallenge/dataset/training_set/',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/My Drive/FSDP2019_DEEP_LEARNING/day30andcodechallenge/dataset/test_set/',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')



classifier.fit_generator(training_set,
                         steps_per_epoch = 50,
                         epochs = 10,#25
                         validation_data = test_set,
                         validation_steps = 200)


Found 710 images belonging to 3 classes.
Found 16 images belonging to 3 classes.
Epoch 1/10





14/50 [=======>......................] - ETA: 10:29 - loss: 0.8695 - acc: 0.5640

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


50/50 [==============================] - 603s 12s/step - loss: 0.6207 - acc: 0.7266 - val_loss: 0.7012 - val_acc: 0.7500
Epoch 2/10
50/50 [==============================] - 225s 5s/step - loss: 0.3660 - acc: 0.8643 - val_loss: 0.8490 - val_acc: 0.7500
Epoch 3/10
50/50 [==============================] - 230s 5s/step - loss: 0.3555 - acc: 0.8577 - val_loss: 0.9790 - val_acc: 0.6875
Epoch 4/10
50/50 [==============================] - 222s 4s/step - loss: 0.2855 - acc: 0.8988 - val_loss: 0.6205 - val_acc: 0.8125
Epoch 5/10
50/50 [==============================] - 225s 5s/step - loss: 0.2241 - acc: 0.9218 - val_loss: 0.7275 - val_acc: 0.8125
Epoch 6/10
50/50 [==============================] - 240s 5s/step - loss: 0.1995 - acc: 0.9312 - val_loss: 0.4853 - val_acc: 0.8125
Epoch 7/10
50/50 [==============================] - 232s 5s/step - loss: 0.1700 - acc: 0.9362 - val_loss: 0.4594 - val_acc: 0.8750
Epoch 8/10
50/50 [==============================] - 227s 5s/step - loss: 0.1435 - acc: 0.9523

In [16]:

# Part 3 - Making new predictions

import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/drive/My Drive/FSDP2019_DEEP_LEARNING/day30andcodechallenge/dataset/test_set/pikachu/000235.jpeg', target_size = (64, 64))

#test_image = image.load_img('/content/drive/My Drive/dataset/single_prediction/000233.png', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices

# {'bulbasaur': 0, 'charmander': 1, 'pikachu': 2}

if int(result[0][0]) == 1:
    prediction = 'Bulbasaur'
elif int(result[0][1]) == 1:
    prediction = 'Charmander'
else:
    prediction = 'Pikachu'

print (prediction)

Pikachu
